In [50]:
import os

In [51]:
%pwd

'/Users/omariko/Documents/GitHub/NLP-Text-Summarizer'

In [52]:
os.chdir("/Users/omariko/Documents/GitHub/NLP-Text-Summarizer/")

In [53]:
%pwd

'/Users/omariko/Documents/GitHub/NLP-Text-Summarizer'

In [54]:
PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_check_point: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [56]:
from TextSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from TextSummarizer.utils.common import read_yaml, create_directories

In [57]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.training_Arguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_check_point = config.model_check_point,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        
        return model_trainer_config

In [58]:
from transformers import (TrainingArguments,
                          Trainer,
                          DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM,
                          AutoTokenizer)
from datasets import load_from_disk
import torch

In [59]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        if (torch.cuda.is_available()):
            device = "cuda"
        else:
            device = "cpu"

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_check_point)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_check_point).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        dataset_samsum = load_from_disk(self.config.data_path)

        trainer_arguments = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs = self.config.num_train_epochs,
            warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_train_batch_size,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            eval_strategy = self.config.eval_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = 1e6,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model = model_pegasus, args = trainer_arguments,
                          tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                          train_dataset = dataset_samsum["test"],
                          eval_dataset = dataset_samsum["validation"])
        trainer.train()

        # Save Model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # Save Tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [60]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-02-27 19:04:53,846: INFO: common: Yaml file: config/config.yaml loadad successfully!]
[2025-02-27 19:04:53,848: INFO: common: Yaml file: params.yaml loadad successfully!]
[2025-02-27 19:04:53,848: INFO: common: Created directory at: artifacts]
[2025-02-27 19:04:53,848: INFO: common: Created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/yl/nmccjv1576l6t_6gf9ghmb480000gn/T/ipykernel_5072/704537527.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus, args = trainer_arguments,


RuntimeError: MPS backend out of memory (MPS allocated: 6.12 GB, other allocations: 2.96 GB, max allowed: 9.07 GB). Tried to allocate 256 bytes on shared pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).